In [ ]:
# Now calculating length of a vector
#train['length'] = train ['dist'] **2
#test['length'] = test ['dist'] **2
# XX, YY, 'ZZ'
train['XX'] = train ['x_0'] * train ['x_1']
train['YY'] = train ['y_0'] * train ['y_1']
train['ZZ'] = train ['z_0'] * train ['z_1']
test['XX'] = test ['x_0'] * test ['x_1']
test['YY'] = test ['y_0'] * test ['y_1']
test['ZZ'] = test ['z_0'] * test ['z_1']


# XY, XZ, 
train['XY'] = train ['x_0'] * train ['y_1']
train['XZ'] = train ['x_0'] * train ['z_1']
test['XY'] = test ['x_0'] * test ['y_1']
test['XZ'] = test ['x_0'] * test ['z_1']


# YX, 'YZ'
train['YX'] = train ['y_0'] * train ['x_1']
train['YZ'] = train ['y_0'] * train ['z_1']
test['YX'] = test ['y_0'] * test ['x_1']
test['YZ'] = test ['y_0'] * test ['z_1']

# ZX, ZY
train['ZX'] = train ['z_0'] * train ['x_1']
train['ZY'] = train ['z_0'] * train ['y_1']
test['ZX'] = test ['z_0'] * test ['x_1']
test['ZY'] = test ['z_0'] * test ['y_1']


# make categorical variables
atom_map = {'H': 0,
            'C': 1,
            'N': 2,
            'O':4}
train['atom_0_cat'] = train['atom_0'].map(atom_map).astype('int')
train['atom_1_cat'] = train['atom_1'].map(atom_map).astype('int')
test['atom_0_cat'] = test['atom_0'].map(atom_map).astype('int')
test['atom_1_cat'] = test['atom_1'].map(atom_map).astype('int')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import os
print(os.listdir("../input"))

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor


In [ ]:
train = pd.read_csv('../input/train.csv', index_col='id')
test = pd.read_csv('../input/test.csv', index_col='id')

In [ ]:
display(train.head())

In [ ]:
train['atom_index_1'].unique()

In [ ]:
#print (test.dtypes)
#print (test.shape)
display(test.head(10))

In [ ]:
ms = pd.read_csv('../input/magnetic_shielding_tensors.csv')
ms.head()

In [ ]:
structures = pd.read_csv('../input/structures.csv')
display(structures.head())

In [ ]:
structures.shape

In [ ]:
pe = pd.read_csv('../input/potential_energy.csv')
display(pe.head())

In [ ]:
pe.shape

In [ ]:
#scalar_coupling_contribution = pd.read_csv('../input/scalar_coupling_contributions.csv')
#display(scalar_coupling_contribution.head())

In [ ]:
# Map the atom structure data into train and test files

def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Engineer a single feature: distance vector between atoms
#  (there's ways to speed this up!)

def dist(row):
    return ( (row['x_1'] - row['x_0'])**2 +
             (row['y_1'] - row['y_0'])**2 +
             (row['z_1'] - row['z_0'])**2 ) ** 0.5

train['dist'] = train.apply(lambda x: dist(x), axis=1)
test['dist'] = test.apply(lambda x: dist(x), axis=1)

In [ ]:
train.head()

In [ ]:
# make categorical variables
atom_map = {'H': 0,
            'C': 1,
            'N': 2,
            'O':4}
train['atom_0_cat'] = train['atom_0'].map(atom_map).astype('int')
train['atom_1_cat'] = train['atom_1'].map(atom_map).astype('int')
test['atom_0_cat'] = test['atom_0'].map(atom_map).astype('int')
test['atom_1_cat'] = test['atom_1'].map(atom_map).astype('int')


In [ ]:
type_map = {'1JHC': 0,
            '2JHH': 1,
            '1JHN': 2,
            '2JHN':3,
           '2JHN':4,
           '2JHC':5,
           '3JHH':6,
           '3JHC':7,
           '3JHN':8}

train['type_cat'] = train['type'].map(type_map).astype('int')
test['type_cat'] = test['type'].map(type_map).astype('int')



In [ ]:
train.head()

In [ ]:
y = train.pop('scalar_coupling_constant')

In [ ]:
X=train.drop(['molecule_name','type','atom_0','atom_1'] ,axis=1)
X.head()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

#y = train.pop('scalar_coupling_constant')
#X = train.drop('molecule_name',axis=1)

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

candidate_max_leaf_nodes = [5,10,15,20, 25]# 50,75, 100,125,150,175,200, 250, 500]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
def get_mae(n_estimators, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=1,n_jobs=-1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

find_n_estimators = [5, 25, 50,75, 100]#,125,150,175,200, 250, 500]
# Write loop to find the ideal tree size from find_n_estimators
for n_estimators in find_n_estimators:
    my_mae = get_mae(n_estimators, train_X, val_X, train_y, val_y)
    print("n_estimators: %d  \t\t Mean Absolute Error:  %d" %(n_estimators, my_mae))

In [ ]:
molecules = train.pop('molecule_name')
test = test.drop('molecule_name', axis=1)

y = train.pop('scalar_coupling_constant')

# Label Encoding
for f in ['type', 'atom_0', 'atom_1']:
    lbl = LabelEncoder()
    lbl.fit(list(train[f].values) + list(train[f].values))
    train[f] = lbl.transform(list(train[f].values))
    test[f] = lbl.transform(list(test[f].values))

In [ ]:
test.head()

In [ ]:
yoof = np.zeros(len(train))
yhat = np.zeros(len(test))

n_splits = 3
gkf = GroupKFold(n_splits=n_splits) # we're going to split folds by molecules

fold = 0
for in_index, oof_index in gkf.split(train, y, groups=molecules):
    fold += 1
    print(f'fold {fold} of {n_splits}')
    X_in, X_oof = train.values[in_index], train.values[oof_index]
    y_in, y_oof = y.values[in_index], y.values[oof_index]
    reg = RandomForestRegressor(n_estimators=10,
                                max_depth=None,
                                min_samples_leaf=3,
                                n_jobs=-1)
    reg.fit(X_in, y_in)
    yoof[oof_index] = reg.predict(X_oof)
    yhat += reg.predict(test)

yhat /= n_splits

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='id')

benchmark = sample_submission.copy()
benchmark['scalar_coupling_constant'] = yhat
benchmark.to_csv('atomic_distance_benchmark.csv')

In [ ]:
plot_data = pd.DataFrame(y)
plot_data.index.name = 'id'
plot_data['yhat'] = yoof
plot_data['type'] = pd.read_csv('../input/train.csv', index_col='id', usecols=['id', 'type'])

def plot_oof_preds(ctype, llim, ulim):
        plt.figure(figsize=(6,6))
        sns.scatterplot(x='scalar_coupling_constant',y='yhat',
                        data=plot_data.loc[plot_data['type']==ctype,
                        ['scalar_coupling_constant', 'yhat']]);
        plt.xlim((llim, ulim))
        plt.ylim((llim, ulim))
        plt.plot([llim, ulim], [llim, ulim])
        plt.xlabel('scalar_coupling_constant')
        plt.ylabel('predicted')
        plt.title(f'{ctype}', fontsize=18)
        plt.show()

plot_oof_preds('1JHC', 0, 250)
plot_oof_preds('1JHN', 0, 100)
plot_oof_preds('2JHC', -50, 50)
plot_oof_preds('2JHH', -50, 50)
plot_oof_preds('2JHN', -25, 25)
plot_oof_preds('3JHC', -25, 100)
plot_oof_preds('3JHH', -20, 20)
plot_oof_preds('3JHN', -15, 15)